In [1]:
# Ackermann steering visualizer with sliders for δ and β
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def draw_car(ax, L=2.5, w=1.5, psi=0.0, delta=0.2, beta=0.0, v=8.0):
    rl = np.array([0.0,  w/2]); rr = np.array([0.0, -w/2])
    fl = np.array([L,  w/2]);  fr = np.array([L, -w/2])
    CoG = np.array([L/2, 0.0])
    Rz = np.array([[np.cos(psi), -np.sin(psi)],[np.sin(psi), np.cos(psi)]])
    def rot(p): return Rz @ p
    rlR, rrR, flR, frR, CoGR = map(rot, [rl, rr, fl, fr, CoG])

    # chassis
    for a,b in [(rlR,flR),(rrR,frR),(rlR,rrR),(flR,frR)]:
        ax.plot([a[0],b[0]],[a[1],b[1]])
    # wheels (rear 0 rad, front δ)
    def wheel(center_local, angle, Lw=0.6):
        cW = rot(center_local); dW = rot(np.array([np.cos(angle), np.sin(angle)]))
        p1, p2 = cW-0.5*Lw*dW, cW+0.5*Lw*dW
        ax.plot([p1[0],p2[0]],[p1[1],p2[1]], linewidth=3)
    wheel(rl,0.0); wheel(rr,0.0); wheel(fl,delta); wheel(fr,delta)

    # CoG + heading ψ and velocity direction ψ+β
    ax.scatter(CoGR[0],CoGR[1],s=50)
    ax.arrow(CoGR[0],CoGR[1],0.9*np.cos(psi),0.9*np.sin(psi),head_width=0.12,length_includes_head=True); ax.text(CoGR[0]+1.0,CoGR[1],r"$\psi$")
    ax.arrow(CoGR[0],CoGR[1],0.9*np.cos(psi+beta),0.9*np.sin(psi+beta),head_width=0.12,length_includes_head=True); ax.text(CoGR[0]+0.8,CoGR[1]+0.4,r"$v$")

    # ICR & rays if δ ≠ 0 (R = L/tanδ)
    if abs(np.tan(delta))>1e-6:
        R = L/np.tan(delta); ICR = rot(np.array([0.0,-R]))
        ax.scatter(ICR[0],ICR[1]); ax.text(ICR[0]+0.1,ICR[1],"ICR")
        for p in [rlR, rrR, flR, frR]:
            ax.plot([p[0],ICR[0]],[p[1],ICR[1]], linestyle="--", alpha=0.5)
    else:
        R = np.inf

    omega = (v/L)*np.cos(beta)*np.tan(delta)   # matches your reference formula
    ax.set_aspect("equal"); ax.grid(True); ax.set_xlabel("X [m]"); ax.set_ylabel("Y [m]")
    ax.set_title(f"Ackermann — δ={np.degrees(delta):.1f}°, β={np.degrees(beta):.1f}°\nR = {R:.2f} m,  ω ≈ {omega:.3f} rad/s")
    ax.set_xlim(-3,6); ax.set_ylim(-4,4)

def visualize(delta_deg=10.0, beta_deg=0.0, L=2.5, w=1.5, v=8.0):
    delta = np.deg2rad(delta_deg); beta = np.deg2rad(beta_deg)
    fig, ax = plt.subplots(figsize=(8,6))
    draw_car(ax, L=L, w=w, psi=0.0, delta=delta, beta=beta, v=v)
    plt.show()

interact(
    visualize,
    delta_deg=FloatSlider(value=10.0, min=-35, max=35, step=0.5, description="Steer δ [deg]"),
    beta_deg=FloatSlider(value=0.0,  min=-20, max=20, step=0.5, description="Sideslip β [deg]"),
    L=FloatSlider(value=2.5, min=1.8, max=3.5, step=0.1, description="Wheelbase L [m]"),
    w=FloatSlider(value=1.5, min=1.2, max=2.0, step=0.05, description="Track w [m]"),
    v=FloatSlider(value=8.0, min=0, max=30, step=0.5, description="Speed v [m/s]"),
);


interactive(children=(FloatSlider(value=10.0, description='Steer δ [deg]', max=35.0, min=-35.0, step=0.5), Flo…

In [2]:
# ============================================================
# Ackermann Steering — Geometry + Dynamics 
# ============================================================
# Author: Ghanashyama Prabhu 💪

import numpy as np
import matplotlib.pyplot as plt

# Try widgets (sliders). If not available, we'll still run a demo.
try:
    from ipywidgets import interact, FloatSlider
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False


# GEOMETRY (kinematic bicycle)

def _draw_car(ax, L=2.5, w=1.5, psi=0.0, delta=0.2, beta=0.0, v=8.0):
    """Draw kinematic bicycle with steering δ and sideslip β."""
    rl = np.array([0.0,  w/2])    # rear-left (body frame)
    rr = np.array([0.0, -w/2])    # rear-right
    fl = np.array([L,  w/2])      # front-left
    fr = np.array([L, -w/2])      # front-right
    CoG = np.array([L/2, 0.0])    # center of gravity (assume lf=lr=L/2)

    Rz = np.array([[np.cos(psi), -np.sin(psi)],
                   [np.sin(psi),  np.cos(psi)]])
    def rot(p): return Rz @ p

    rlR, rrR, flR, frR, CoGR = map(rot, [rl, rr, fl, fr, CoG])

    # chassis
    for a, b in [(rlR, flR), (rrR, frR), (rlR, rrR), (flR, frR)]:
        ax.plot([a[0], b[0]], [a[1], b[1]], 'k-')

    # wheels (rear aligned with body, front steered by δ)
    def wheel(center_local, ang_local, Lw=0.6):
        cW = rot(center_local)
        dW = rot(np.array([np.cos(ang_local), np.sin(ang_local)]))
        p1, p2 = cW - 0.5*Lw*dW, cW + 0.5*Lw*dW
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'b-', linewidth=3)

    wheel(rl, 0.0); wheel(rr, 0.0)
    wheel(fl, delta); wheel(fr, delta)

    # CoG + heading ψ (black) and velocity direction v (ψ+β, green)
    ax.scatter(CoGR[0], CoGR[1], s=50, c='k')
    ax.arrow(CoGR[0], CoGR[1], 0.9*np.cos(psi), 0.9*np.sin(psi),
             head_width=0.12, length_includes_head=True)
    ax.text(CoGR[0]+1.05*np.cos(psi), CoGR[1]+1.05*np.sin(psi), r"$\psi$")
    ax.arrow(CoGR[0], CoGR[1], 0.9*np.cos(psi+beta), 0.9*np.sin(psi+beta),
             head_width=0.12, length_includes_head=True)
    ax.text(CoGR[0]+0.9*np.cos(psi+beta), CoGR[1]+0.9*np.sin(psi+beta)+0.15, r"$v$")

    # ICR and rays (R = L / tan δ)
    if abs(np.tan(delta)) > 1e-6:
        R = L / np.tan(delta)
        ICR = rot(np.array([0.0, -R]))
        ax.scatter(ICR[0], ICR[1], c='r')
        ax.text(ICR[0]+0.1, ICR[1], "ICR", color='r')
        for p in [rlR, rrR, flR, frR]:
            ax.plot([p[0], ICR[0]], [p[1], ICR[1]], 'r--', alpha=0.5)
    else:
        R = np.inf

    # Approx yaw rate from boxed kinematic-with-β relation: ω ≈ (v/L) cosβ tanδ
    omega = (v / L) * np.cos(beta) * np.tan(delta)

    ax.set_aspect('equal'); ax.grid(True)
    ax.set_xlim(-3, 6); ax.set_ylim(-4, 4)
    ax.set_xlabel("X [m]"); ax.set_ylabel("Y [m]")
    ax.set_title(f"Ackermann geometry — δ={np.degrees(delta):.1f}°, β={np.degrees(beta):.1f}°\n"
                 f"R = {R:.2f} m,  ω ≈ {omega:.3f} rad/s")

def visualize_geometry(delta_deg=10.0, beta_deg=0.0, L=2.5, w=1.5, v=8.0):
    """One-shot geometry figure (works even without widgets)."""
    delta = np.deg2rad(delta_deg); beta = np.deg2rad(beta_deg)
    fig, ax = plt.subplots(figsize=(8, 6))
    _draw_car(ax, L=L, w=w, psi=0.0, delta=delta, beta=beta, v=v)
    plt.show()

# DYNAMICS (nonlinear bicycle model)

def _slip_angles(vx, vy, r, lf, lr, delta):
    eps = 1e-3
    vx_s = vx if abs(vx) > eps else (eps if vx >= 0 else -eps)
    af = np.arctan2(vy + lf*r, vx_s) - delta
    ar = np.arctan2(vy - lr*r, vx_s)
    return af, ar

def _tire_forces_linear(af, ar, Cf, Cr):
    # Small-slip lateral forces
    return -Cf * af, -Cr * ar

def _dyn_step(state, delta, Fx_f, Fx_r, p, dt):
    # State: [x, y, psi, vx, vy, r]
    x, y, psi, vx, vy, r = state
    m, Iz, L, lf, lr, Cf, Cr = p["m"], p["Iz"], p["L"], p["lf"], p["lr"], p["Cf"], p["Cr"]

    af, ar = _slip_angles(vx, vy, r, lf, lr, delta)
    Fy_f, Fy_r = _tire_forces_linear(af, ar, Cf, Cr)

    # Body-frame dynamics
    ax = (Fx_f*np.cos(delta) - Fy_f*np.sin(delta) + Fx_r)/m + r*vy
    ay = (Fx_f*np.sin(delta) + Fy_f*np.cos(delta) + Fy_r)/m - r*vx
    rr = (lf*(Fy_f*np.cos(delta) + Fx_f*np.sin(delta)) - lr*Fy_r)/Iz

    # Integrate speeds (Euler)
    vx += ax*dt; vy += ay*dt; r += rr*dt

    # Pose kinematics to world
    x  += (vx*np.cos(psi) - vy*np.sin(psi)) * dt
    y  += (vx*np.sin(psi) + vy*np.cos(psi)) * dt
    psi += r * dt

    return np.array([x, y, psi, vx, vy, r])

def simulate_dynamic(delta_deg=10.0, v=12.0, T=10.0, dt=0.01, beta0_deg=0.0, params=None):
    """Run a constant-δ simulation and try to hold speed at v with a crude throttle."""
    if params is None:
        params = dict(m=1500.0, Iz=2250.0, L=2.8, lf=1.4, lr=1.4,
                      Cf=8e4, Cr=9e4, mu=0.9, g=9.81)

    delta = np.deg2rad(delta_deg)
    N = int(T / dt)
    t = np.arange(N) * dt

    # Initial state with optional initial sideslip β0
    vx0 = max(0.1, 0.1*v)
    vy0 = np.tan(np.deg2rad(beta0_deg)) * vx0
    state = np.array([0.0, 0.0, 0.0, vx0, vy0, 0.0])
    X = np.zeros((N, 6)); beta = np.zeros(N)

    # Simple proportional "throttle" to track v_x → v
    k_throttle = 500.0
    for k in range(N):
        vx = state[3]
        Fx_total = params["m"] * k_throttle * (v - vx) / params["m"]
        Fx_f, Fx_r = 0.4*Fx_total, 0.6*Fx_total
        state = _dyn_step(state, delta, Fx_f, Fx_r, params, dt)
        X[k, :] = state
        beta[k] = np.arctan2(state[4], state[3])

    return t, X, beta

# Plotting
def plot_xy(t, X):
    plt.figure(figsize=(6, 6))
    plt.plot(X[:, 0], X[:, 1])
    plt.axis('equal'); plt.grid(True)
    plt.xlabel("x [m]"); plt.ylabel("y [m]")
    plt.title("Trajectory (dynamic bicycle)")
    plt.show()

def plot_yawrate(t, X):
    plt.figure(figsize=(6, 4))
    plt.plot(t, X[:, 5])
    plt.grid(True)
    plt.xlabel("time [s]"); plt.ylabel("r [rad/s]")
    plt.title("Yaw rate r(t)")
    plt.show()

def plot_beta(t, beta):
    plt.figure(figsize=(6, 4))
    plt.plot(t, beta)
    plt.grid(True)
    plt.xlabel("time [s]"); plt.ylabel("β [rad]")
    plt.title("Sideslip β(t)")
    plt.show()

def plot_speed(t, X, v_ref):
    plt.figure(figsize=(6, 4))
    plt.plot(t, X[:, 3], label="v_x")
    plt.hlines([v_ref], t[0], t[-1], linestyles="--", label="target v")
    plt.grid(True); plt.legend()
    plt.xlabel("time [s]"); plt.ylabel("speed [m/s]")
    plt.title("Speed tracking")
    plt.show()

def simulate_and_plot(delta_deg=10.0, v=12.0, T=10.0, dt=0.01, beta0_deg=0.0):
    t, X, beta = simulate_dynamic(delta_deg=delta_deg, v=v, T=T, dt=dt, beta0_deg=beta0_deg)
    plot_xy(t, X); plot_yawrate(t, X); plot_beta(t, beta); plot_speed(t, X, v)


def _run_widgets():
    # Geometry sliders
    print("== Geometry view ==")
    interact(
        visualize_geometry,
        delta_deg=FloatSlider(value=10.0, min=-35.0, max=35.0, step=0.5, description="Steer δ [deg]"),
        beta_deg=FloatSlider(value=0.0,  min=-20.0, max=20.0, step=0.5, description="Sideslip β [deg]"),
        L=FloatSlider(value=2.5, min=1.8, max=3.5, step=0.1, description="Wheelbase L [m]"),
        w=FloatSlider(value=1.5, min=1.2, max=2.0, step=0.05, description="Track w [m]"),
        v=FloatSlider(value=8.0, min=0.0, max=30.0, step=0.5, description="Speed v [m/s]"),
    )

    # Dynamics sliders
    print("== Dynamic view ==")
    interact(
        simulate_and_plot,
        delta_deg=FloatSlider(value=10.0, min=-30.0, max=30.0, step=0.5, description="Steer δ [deg]"),
        beta0_deg=FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.5, description="Init β0 [deg]"),
        v=FloatSlider(value=12.0, min=0.0, max=30.0, step=0.5, description="Speed v [m/s]"),
        T=FloatSlider(value=10.0, min=3.0, max=20.0, step=1.0, description="Duration T [s]"),
        dt=FloatSlider(value=0.01, min=0.005, max=0.05, step=0.005, description="dt [s]"),
    )

# Run interactives if possible; otherwise, run a demo
if HAS_WIDGETS:
    _run_widgets()
else:
    print("ipywidgets not found — running demo. You can still call functions manually.")
    # Geometry demo
    visualize_geometry(delta_deg=12.0, beta_deg=3.0, L=2.6, w=1.5, v=10.0)
    # Dynamics demo
    simulate_and_plot(delta_deg=10.0, v=12.0, T=8.0, dt=0.01, beta0_deg=0.0)

# Usage without widgets:
#   visualize_geometry(15, 5)                   # geometry: δ=15°, β=5°
#   simulate_and_plot(delta_deg=8, v=10, T=12)  # dynamics with constant δ and speed target


== Geometry view ==


interactive(children=(FloatSlider(value=10.0, description='Steer δ [deg]', max=35.0, min=-35.0, step=0.5), Flo…

== Dynamic view ==


interactive(children=(FloatSlider(value=10.0, description='Steer δ [deg]', max=30.0, min=-30.0, step=0.5), Flo…